# Weather data is processed in this notebook.

I am using 2 csv files which I downloaded from https://openweathermap.org.
The files have hourly value records.
The files 'Skg_Wheather_History_1997_2021.csv' are essentially the history of the weather of Thessaloniki for the period 1997 until July 2021.
While the file 'Skg_Wheather_History_2021_2022.csv' is the weather history from August 2021 to January 2022.

In [1]:
import pandas as pd
Project_Path='Local Path'


I read and convert the files as a DataFrame and keep only the columns I need.
- 'dt_iso' is the DateTime of each recording.
- 'temp' is the temperature in degrees Celsius
- 'Humidity' is the hydration expressed in values from 0 to 100
In the data there were a few duplicate entries, I deleted them as I will then use the 'dt_iso' column as a key to merge
Then I merged the two DataFrames into one.

In [17]:
Weather_Data_Part_1 = pd.read_csv(Project_Path+ '/Data/OpenWheather/Skg_Wheather_History_2016_2021.csv', sep=';',low_memory=False)
Weather_Data_Part_2 = pd.read_csv(Project_Path+ '/Data/OpenWheather/Skg_Wheather_History_2021_2022.csv', sep=',',low_memory=False)

Weather_Data_Part_1=Weather_Data_Part_1[["dt_iso", "temp","humidity"]]
Weather_Data_Part_2=Weather_Data_Part_2[["dt_iso", "temp","humidity"]]

Weather_Data_Part_1.drop_duplicates(subset ="dt_iso",
                     keep ='first', inplace = True)
Weather_Data_Part_2.drop_duplicates(subset ="dt_iso",
                     keep ='first', inplace = True)

Weather_Data = pd.concat([Weather_Data_Part_1, Weather_Data_Part_2],ignore_index=True)

Then I convert the format of the values of column 'dt_iso'
from <1979-01-01 00:00:00 +0000 UTC> to <1979-01-01 00:00> and name this new column 'Key'.
I am doing this conversion because I will use the 'Key' to merge the weather data with the rest of the data afterwards.

In [18]:
Final_Weather_Data=Weather_Data[["dt_iso", "temp","humidity"]]
Weather_List=Final_Weather_Data.values.tolist()
Dt=[]
for i in range (0,len(Weather_List)):
    Dt.append(Weather_List[i][0].split(' '))
Date_Df = pd.DataFrame(Dt, columns=["Date","Time","Np1","Np2"])

Time_List=Date_Df['Time'].values.tolist()
Time=[]
for i in range (0,len(Time_List)):
    String=str(Time_List[i])
    X=String[0:5]
    Time.append(X)

Final_Weather_Data.insert(0, "Date", Date_Df['Date'], True)
Final_Weather_Data.insert(1, "Time", Time, True)
Final_Weather_Data=Final_Weather_Data.drop(['dt_iso'], axis=1)

Final_Weather_Data['Key']=Final_Weather_Data['Date'].map(str)+ ' ' +Final_Weather_Data['Time'].map(str)
Final_Weather_Data=Final_Weather_Data.drop(['Time'], axis=1)
Final_Weather_Data=Final_Weather_Data.drop(['Date'], axis=1)

I keep the hourly values of temperature and humidity only for the time I need

In [19]:
Final_Weather_Data = Final_Weather_Data[~(Final_Weather_Data['Key'] < '2017-12-31')]
Final_Weather_Data = Final_Weather_Data[~(Final_Weather_Data['Key'] >= '2021-10-02')]
Final_Weather_Data=Final_Weather_Data.reset_index()
Final_Weather_Data=Final_Weather_Data.drop(['index'], axis=1)

Final_Weather_Data['temp']=Final_Weather_Data['temp'].astype(float)
Final_Weather_Data['humidity']=Final_Weather_Data['humidity'].astype(float)

I convert the temperature and humidity values so that the values for a particular hour are the 6 hour average.
The 6 hours I use to get the average time is the time I have for the previous 5.
I then save the Dataframe 'Final_Weather_Data' to use later

In [21]:
List=Final_Weather_Data.values.tolist()
New_Temp=[]
New_Humidity=[]
Index_List=[0,1,2,3,4] 
for i in range(0,len(List)):
    Temp_Sum=0
    Humidity_Sum=0
    Humidity_Sum=0
    # For the first 4 hourly values of the DataFrame since I don't know the previous ones
    # I leave as is. Also, I don't need them at the specific times given the data
    # starts from January 1, 2018
    if i in Index_List:
        New_Temp.append(List[i][0])
        New_Humidity.append(List[i][1])  
    else:
        for j in range (0,6):
            Temp_Sum=Temp_Sum+List[i-j][0]
            Humidity_Sum=Humidity_Sum+List[i-j][1]
        
        New_Temp.append(Temp_Sum/8)
        New_Humidity.append(Humidity_Sum/8)


Final_Weather_Data=Final_Weather_Data.drop(['humidity'], axis=1)
Final_Weather_Data=Final_Weather_Data.drop(['temp'], axis=1)
Final_Weather_Data.insert(1, "temp", New_Temp, True)
Final_Weather_Data.insert(2, "humidity", New_Humidity, True)  


Final_Weather_Data.to_csv(Project_Path+ '/Data/Final_Weather_Data.csv')
Final_Weather_Data

,Key,temp,humidity
0,2017-12-31 00:00,5.96000,49.000
1,2017-12-31 01:00,5.78000,45.000
2,2017-12-31 02:00,5.51000,45.000
3,2017-12-31 03:00,5.31000,43.000
4,2017-12-31 04:00,4.57000,52.000
...,...,...,...
32899,2021-10-01 19:00,13.73500,49.000
32900,2021-10-01 20:00,13.34875,51.125
32901,2021-10-01 21:00,13.01375,51.250
32902,2021-10-01 22:00,12.75750,50.250
